**Un dashboard qui facilite la surveillance et la prise de décision.**

Domaine : énergies et météo

Ce dashboard doit permetre de savoir quand allumer la source de chaleur pour optimiser la chauffe du foyer.

*"j'ai besoin de savoir quand allumer mon poelle, à quelle température, pendant combien de temps, en prévision de la météo externe pour maintenir une température agréable dans ma maison en fonction de la qualité du bâti"*

***Je commence par la création de mon (.venv).***
puis créer la variable de l'environnement à la racine du dossier ".env"

Dans le terminal :

(data_visualisation) C:\Users\MAF\Documents\data_visualisation>.\Scripts\activate

pip list

(.venv) PS C:\Users\MAF\Documents\data_visualisation> python -m pip install -r .\requirements.txt

**La liste qu'on doit avoir issue du requirement.txt**

pandas
numpy
matplotlib
seaborn
plotly
python-dotenv
openpyxl
pyarrow
scipy

***Chargement des bibliothèques***

In [1]:
import os, pandas as pd, numpy as np
from dotenv import load_dotenv
import plotly.express as px
load_dotenv()
DATA_DIR = os.getenv("DATA_DIR","./data")
pd.set_option("display.max_columns", None)

**trier les fichiers csv utilisables de ceux qui le ne sont pas. (j'ai essayé les bases de données SQL mais très lourdes, j'ai trouvé un csv plus léger, à utiliser en cas de dernier recour)**

Liste des fichier disponibles.

In [ ]:
import glob
csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
csv_files


[]

In [ ]:
%%time
df_sample = pd.read_csv(path, sep=";", nrows=1000)
df_sample.head(20)


df: shape=(0, 0)
df2: shape=(0, 0)
df3: shape=(0, 0)


""


***Chargement des données***

Mes trois familles de datasets (météo, intérieur, bâtiment).

Variables :

Température extérieure horaire (temp_ext)

Humidité, vent, précipitation, couverture nuageuse

Heure, jour, saison

Météo (heure heure par heure)

In [ ]:
# Chargement des fichiers CSV dans la variable `data`
# Utilise la variable d'environnement DATA_DIR si définie, sinon ./data puis le répertoire courant
data_dir = os.getenv("DATA_DIR", "./data")
if not os.path.isdir(data_dir):
    data_dir = "."

csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(".csv")]

if not csv_files:
    raise FileNotFoundError(f"Aucun fichier .csv trouvé dans {data_dir} ou répertoire courant.")

dfs = []
for fp in csv_files:
    try:
        df = pd.read_csv(fp, low_memory=False)
    except Exception as e:
        print(f"Erreur en lisant {fp}: {e}")
        continue

    # Ajoute une colonne source pour tracer l'origine des données
    df["__source_file__"] = os.path.basename(fp)

    # Tente de détecter et convertir une colonne de date/heure courante
    for col in ["date", "datetime", "time", "timestamp", "jour", "heure"]:
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce", infer_datetime_format=True)
                # si conversion OK, place en index et normalise le nom
                if df[col].notna().any():
                    df = df.rename(columns={col: "datetime"})
                    df = df.set_index("datetime")
                break
            except Exception:
                pass

    dfs.append(df)

# Concatène tous les fichiers
data = pd.concat(dfs, axis=0, ignore_index=False, sort=False)

# Affiche résumé
print(f"{len(dfs)} fichier(s) chargés. Dimensions combinées: {data.shape}")
data.head()

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

***Analyses et visualisations***

(le visuel que je souhaite utiliser pour représenter l'occurence de chauffage est un Time-Series)

In [1]:
data.plot(y=['tavg', 'tmin', 'tmax'], title="Températures Val-d’Oise")
plt.xlabel("Date")
plt.ylabel("Température (°C)")
plt.grid(True)
plt.show()


NameError: name 'data' is not defined